Note: not able to get this to work... it seems to require a pretrained model setting that I'm not able to replicate


In [1]:
from google.colab import drive
#drive.mount('/content/gdrive')

drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#!pip install pytorch-pretrained-bert

In [2]:
import sys
print(sys.path)
#from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE

['', '/env/python', '/usr/lib/python36.zip', '/usr/lib/python3.6', '/usr/lib/python3.6/lib-dynload', '/usr/local/lib/python3.6/dist-packages', '/usr/lib/python3/dist-packages', '/usr/local/lib/python3.6/dist-packages/IPython/extensions', '/root/.ipython']


In [3]:
import csv
import logging
import os
import random
import sys

import numpy as np
import torch
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

#load the forked "pretrained_bert_model" settings included in the drg repo (use this for bertviz stuff too...)
drg_repo_path = '/content/gdrive/My Drive/humor_style_transfer/transformer-drg-style-transfer'
if not drg_repo_path in sys.path:
  sys.path.append(drg_repo_path)

from pytorch_pretrained_bert.file_utils import PYTORCH_PRETRAINED_BERT_CACHE
from pytorch_pretrained_bert.modeling import BertForSequenceClassification, BertConfig, WEIGHTS_NAME, CONFIG_NAME
#from pytorch_pretrained_bert.tokenization import BertTokenizer
from pytorch_pretrained_bert.optimization import BertAdam, warmup_linear #(NOTE: IN THE NEWEST VERSION, 'WARMUP_LINEAR' = ARG FOR BERTADAM)

# this part was misbehaving in earlier versions.
from bertviz.bertviz import attention, visualization
from bertviz.bertviz.pytorch_pretrained_bert import BertModel, BertTokenizer

Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.
Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex.


In [0]:
logger = logging.getLogger(__name__)
bert_classifier_model_dir = "./bert_classifier/" ## Path of BERT classifier model path (unavailable ???)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
logger.info("device: {}, n_gpu {}".format(device, n_gpu))

In [0]:
# file paths
data_dir = "./gdrive/My Drive/humor_style_transfer/style-transfer-data/Sentiment-and-Style-Transfer/data"
dataset = "yelp" # amazon / yelp / imagecaption
train_0 = os.path.join(data_dir ,"./{}/sentiment.train.0".format(dataset))
train_1 = os.path.join(data_dir,"./{}/sentiment.train.1".format(dataset))
test_0 = os.path.join(data_dir,"./{}/sentiment.test.0".format(dataset))
test_1 = os.path.join(data_dir,"./{}/sentiment.test.1".format(dataset))
dev_0 = os.path.join(data_dir,"./{}/sentiment.dev.0".format(dataset))
dev_1 = os.path.join(data_dir,"./{}/sentiment.dev.1".format(dataset))
reference_0 = os.path.join(data_dir,"./{}/reference.0".format(dataset))
reference_1 = os.path.join(data_dir,"./{}/reference.1".format(dataset))

In [0]:
# file paths
data_dir = "./gdrive/My Drive/humor_style_transfer/style-transfer-data/Sentiment-and-Style-Transfer/data"
dataset = "yelp" # amazon / yelp / imagecaption
train_0_out = os.path.join(data_dir ,"./{}/processed_files_with_bert_with_best_head/sentiment.train.0".format(dataset))
train_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment.train.1".format(dataset))
test_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment.test.0".format(dataset))
test_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment.test.1".format(dataset))
dev_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment.dev.0".format(dataset))
dev_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/sentiment.dev.1".format(dataset))
reference_0_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/reference.0".format(dataset))
reference_1_out = os.path.join(data_dir,"./{}/processed_files_with_bert_with_best_head/reference.1".format(dataset))

In [7]:
## Model for performing Classification

#[use this after pretrained model found?]
#model_cls = BertForSequenceClassification.from_pretrained(bert_classifier_model_dir, num_labels=2) 

#I think this is downloading bert model - save it later.
model_cls = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model_cls.to(device)
model_cls.eval()

100%|██████████| 231508/231508 [00:00<00:00, 614685.85B/s]


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1, inplace=False)
            )
    

In [8]:
## Model to get the attention weights of all the heads

### === the below seems not to be working properly... I don't understand what the required output format should be ===
#model = BertModel.from_pretrained(bert_classifier_model_dir)
model = BertModel.from_pretrained('bert-base-uncased')

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
model.to(device)
model.eval()

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): BertLayerNorm()
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): BertLayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (intermediate): BertIntermediate(
          (dense): Linear(in_features

In [0]:
max_seq_len=70 # Maximum sequence length 
sm = torch.nn.Softmax(dim=-1) ## Softmax over the batch

In [0]:
common_words=['is','are','was','were','has','have','had','a','an','the','this','that','these','those','there','how','i','we',
             'he','she','it','they','them','their','his','him','her','us','our', 'and','in','my','your','you', 'will', 'shall']
common_words_tokens = tokenizer.convert_tokens_to_ids(common_words)
not_to_remove_ids = tokenizer.convert_tokens_to_ids(["[CLS]","[SEP]", ".", "?", "!"])
not_to_remove_ids += common_words_tokens

In [0]:
def read_file(file_path):
    with open(file_path) as fp:
        data = fp.read().splitlines()
    return data

In [0]:
def create_output_file(original_sentences,processed_sentences, output_file, sentiment="<POS>"):
    with open(output_file,"w") as fp:
        for sen1,sen2 in zip(original_sentences,processed_sentences):
            if sen1 != None and sen2 != None:
                str1 = sentiment + " <CON_START> " + sen2 + " <START> " + sen1 + " <END>\n"
                fp.write(str1)

In [0]:
#call from lower part: #create_ref_output_file(ref_1_data, ref_1_out_sen, reference_1_out, sentiment="<NEG>")
def create_ref_output_file(processed_sentences, output_file, sentiment="<POS>"):
    with open(output_file,"w") as fp:
        for sen in tqdm(processed_sentences):
            if sen != None:
                str1 = sentiment + " <CON_START> " + sen + " <START>\n"
                fp.write(str1)

In [0]:
def concate_files(inp_files, out_files):
    with open(out_files,"w") as fp:
        for file in inp_files:
            with open(file) as f:
                for line in f:
                    fp.write(line)

In [0]:
def run_attn_examples(input_sentences, layer, head, bs=128):
    """
    Returns Attention weights for selected Layer and Head along with ids and tokens
    of the input_sentence
    """
    ids = []
    ids_to_decode = [None for k in range(len(input_sentences))]
    tokens_to_decode = [None for k in range(len(input_sentences))]
    segment_ids = []
    input_masks = []
    attention_weights = [None for z in input_sentences]
    ## BERT pre-processing
    for j,sen in enumerate(tqdm(input_sentences)):
        
        text_tokens = tokenizer.tokenize(sen)
        if len(text_tokens) >= max_seq_len-2:
            text_tokens = text_tokens[:max_seq_len-4]
        tokens = ["[CLS]"] + text_tokens + ["[SEP]"]
        tokens_to_decode[j] = tokens
        temp_ids = tokenizer.convert_tokens_to_ids(tokens)
        ids_to_decode[j] = temp_ids
        input_mask = [1] * len(temp_ids)
        segment_id = [0] * len(temp_ids)
        padding = [0] * (max_seq_len - len(temp_ids))
        
        
        temp_ids += padding
        input_mask += padding
        segment_id += padding
        
        ids.append(temp_ids)
        input_masks.append(input_mask)
        segment_ids.append(segment_id)
    
    # Convert Ids to Torch Tensors
    ids = torch.tensor(ids) 
    segment_ids = torch.tensor(segment_ids)
    input_masks = torch.tensor(input_masks)
    
    steps = len(ids) // bs
    
    for i in trange(steps+1):
        if i == steps:
            temp_ids = ids[i * bs : len(ids)]
            temp_segment_ids = segment_ids[i * bs: len(ids)]
            temp_input_masks = input_masks[i * bs: len(ids)]
        else:
            temp_ids = ids[i * bs : i * bs + bs]
            temp_segment_ids = segment_ids[i * bs: i * bs + bs]
            temp_input_masks = input_masks[i * bs: i * bs + bs]
        
        temp_ids = temp_ids.to(device)
        temp_segment_ids = temp_segment_ids.to(device)
        temp_input_masks = temp_input_masks.to(device)
        with torch.no_grad():
          _, _, attn = model(temp_ids, temp_segment_ids, temp_input_masks)
          #WARNING: changed model; produces 2 outputs not 3 (???)
            ## what does the below do ??
             #_, _, attn = model(temp_ids, temp_segment_ids, temp_input_masks)
        # Concate Attention weights
        for j in range(len(attn[layer]['attn_probs'])):
            attention_weights[i * bs + j] = (attn[layer]['attn_probs'][j][head][0]).to('cpu')
    
    return attention_weights, ids_to_decode, tokens_to_decode

In [0]:
def prepare_data(aw, ids_to_decode, tokens_to_decode):
    out_sen = [None for i in range(len(aw))]
    for i in trange(len(aw)):
        #topv, topi = aw[i].topk(len(inps_tokens[i]))
        topv, topi = aw[i].topk(ids_to_decode[i].index(0))
        topi = topi.tolist()
        topv = topv.tolist()
        #print(i,train_0[i])
        #print(tokens_to_decode[i])
        #print("Original Top Indexes = {}".format(topi))
        topi = [topi[j] for j in range(len(topi)) if ids_to_decode[i][topi[j]] not in not_to_remove_ids] # remove noun and common words
        #print("After removing Nouns = {}".format(topi))
        topi = [topi[j] for j in range(len(topi)) if "##" not in tokens_to_decode[i][topi[j]]] # Remove half words
        #print("After removing Half-words = {}".format(topi))

        if (len(topi) < 4 and len(topi) > 0):
            topi = [topi[0]]
        elif(len(topi) < 8):
            topi = topi[:2]
        else:
            topi = topi[:3]

        #print("Final Topi = {}".format(topi))
        final_indexes = []
        count = 0
        count1 = 0
        #print(ids_to_decode[i], tokens_to_decode[i])
        while ids_to_decode[i][count] != 0:
            if count in topi:
                while ids_to_decode[i][count + count1 + 1] != 0:
                    if "##" in tokens_to_decode[i][count + count1 + 1]:
                        count1 += 1
                    else:
                        break
                count += count1
                count1 = 0
            else:
                final_indexes.append(ids_to_decode[i][count])
            count += 1

        #print(final_indexes)
        temp_out_sen = tokenizer.convert_ids_to_tokens(final_indexes)
        temp_out_sen = " ".join(temp_out_sen).replace(" ##", "").replace("[CLS]","").replace("[SEP]","")
        #print(temp_out_sen, "\n\n")
        out_sen[i] = temp_out_sen.strip()
    
    return out_sen

In [0]:
train_0_data = read_file(train_0)
train_1_data = read_file(train_1)
dev_0_data = read_file(dev_0)
dev_1_data = read_file(dev_1)
test_0_data = read_file(test_0)
test_1_data = read_file(test_1)
ref_0_data = read_file(reference_0)
ref_1_data = read_file(reference_1)

In [18]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_0_data, layer=9, head=7, bs=128)
train_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_0_data, train_0_out_sen, train_0_out, sentiment="<NEG>")

100%|██████████| 177218/177218 [00:04<00:00, 42972.92it/s]


In [19]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(train_1_data, layer=9, head=7, bs=128)
train_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(train_1_data, train_1_out_sen, train_1_out, sentiment="<POS>")

100%|██████████| 266041/266041 [00:05<00:00, 47816.04it/s]


In [20]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_0_data, layer=9, head=7, bs=128)
dev_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_0_data, dev_0_out_sen, dev_0_out, sentiment="<NEG>")

100%|██████████| 2000/2000 [00:00<00:00, 40827.83it/s]


In [21]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(dev_1_data, layer=9, head=7, bs=128)
dev_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(dev_1_data, dev_1_out_sen, dev_1_out, sentiment="<POS>")

100%|██████████| 2000/2000 [00:00<00:00, 38621.76it/s]


In [22]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_1_data, layer=9, head=7, bs=128)
test_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_1_data, test_1_out_sen, test_1_out, sentiment="<POS>")

100%|██████████| 500/500 [00:00<00:00, 32420.49it/s]


In [23]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(test_0_data, layer=9, head=7, bs=128)
test_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
create_output_file(test_0_data, test_0_out_sen, test_0_out, sentiment="<NEG>")

100%|██████████| 500/500 [00:00<00:00, 33033.82it/s]


In [25]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_1_data, layer=9, head=7, bs=128)
ref_1_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
#create_ref_output_file(ref_1_data, ref_1_out_sen, reference_1_out, sentiment="<NEG>")
#changing from above ^^: ref file takes only processed sentences
create_ref_output_file(ref_1_out_sen, reference_1_out, sentiment="<NEG>")

100%|██████████| 500/500 [00:00<00:00, 521290.58it/s]


In [26]:
aw, ids_to_decode, tokens_to_decode = run_attn_examples(ref_0_data, layer=9, head=7, bs=128)
ref_0_out_sen = prepare_data(aw, ids_to_decode, tokens_to_decode)
#create_ref_output_file(ref_0_data, ref_0_out_sen, reference_0_out, sentiment="<POS>")
#changing from above ^^: ref file takes only processed sentences
create_ref_output_file(ref_0_out_sen, reference_0_out, sentiment="<POS>")

100%|██████████| 500/500 [00:00<00:00, 213733.39it/s]
